![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodales RAG
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

In [ ]:
#@title 📂 Dokumente, Bilder, Modul { display-mode: "form" }
!rm -rf files
!mkdir files

# --- Texte
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/roboter.txt -o files/roboter.txt

# --- Bilder
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/retro_robot.jpg -o files/retro_robot.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/hedra_cyborg.png -o files/hedra_cyborg.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/apfel.png -o files/apfel.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/zwei_roboter.png -o zwei_roboter.png

# --- Python-Modul
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/01%20ipynb/multimodal_rag_modul.py -o multimodal_rag_modul.py

# 1 | RAG-Prozess
---

In dem entwickelten **multimodalen Retrieval-Augmented-Generation (RAG)**-System werden sowohl Text- als auch Bilddaten verarbeitet.
Für die **Bildverarbeitung** kommt ein zweistufiger Ansatz zum Einsatz:

1. **Image-Embedding:**
   Das Bild wird in einen hochdimensionalen Vektorraum eingebettet, um visuelle Merkmale wie Formen, Farben und Strukturen numerisch zu repräsentieren.

2. **Textbeschreibung und Text-Embedding:**
   Zusätzlich wird mit *gpt-4o-mini* eine sprachliche Beschreibung des Bildes erzeugt. Diese Beschreibung wird anschließend in ein Text-Embedding überführt, um semantische Informationen textbasiert nutzbar zu machen.

---

**Vorteile des Ansatzes:**

* **Erweiterte semantische Repräsentation:**
  Durch die Kombination von visuellen und sprachlichen Embeddings werden sowohl konkrete als auch konzeptuelle Eigenschaften eines Bildes abgebildet.

* **Verbesserte Retrieval-Qualität:**
  Textbasierte Suchanfragen können nicht nur über visuelle Ähnlichkeiten, sondern auch über die semantisch beschriebene Bedeutung der Bilder beantwortet werden.

* **Höhere Interpretierbarkeit:**
  Die generierte Bildbeschreibung ermöglicht eine transparente Nachvollziehbarkeit der zugrunde liegenden Repräsentationen und unterstützt bei der Evaluierung der Ergebnisse.

* **Gute Erweiterbarkeit:**
  Das Verfahren ist modular aufgebaut und lässt sich leicht um weitere Modalitäten wie Audio oder Video ergänzen.


<p><font color='black' size="5">
RAG-Prozess für Texte
</font></p>

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_process.png" width="600" alt="Avatar">

<p><font color='black' size="5">
RAG-Prozess für Bilder
</font></p>

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_process_03.png" width="600" alt="Avatar">

# 2 | Modul `multimodal_rag`
---

Python-Modul für ein **funktionales multimodales RAG-System**, das Text- und Bilddokumente in einer einheitlichen Vektordatenbank verwaltet und durchsucht.

**Modalitätsrichtungen**

| Eingabe (Query) | Ausgabe (Antwort) | Beispiel / Beschreibung | Status |
|-----------------|-------------------|-------------------------|--------|
| **Text → Text** | Textbasierte Frage führt zu Textantwort | Klassisches RAG-System (z.B. Chatbot, Q&A) | ✅ |
| **Text → Bild** | Textanfrage findet relevante Bilder | "Zeige mir Roboter-Bilder" | ✅ |
| **Bild → Text** | Bildanalyse oder Captioning | "Was ist auf diesem Foto zu sehen?" | ✅ |
| **Bild → Bild** | Bildretrieval oder visuelle Transformation | "Finde ähnliche Bilder" | ✅ |
| **Text + Bild → Text** | Kombination zur Textgenerierung | "Welche Informationen enthält dieses Diagramm?" | ❌ |
| **Text + Bild → Bild** | Bedingte Bildgenerierung | "Mach aus diesem Bild eine Winterversion" | ❌ |

**Hauptvorteile**

1. **Funktionale Architektur**: Klare Trennung von Konfiguration, Komponenten und Funktionen
2. **Einheitliche Datenbank**: ChromaDB mit separaten Collections für Text und Bilder
3. **Hybride Suche**: Text-Embeddings (OpenAI) + Bild-Embeddings (CLIP)
4. **Flexible Konfiguration**: Alle Parameter über RAGConfig anpassbar
5. **Bildbeschreibung**: Zu jedem Bild wird zusätzlich eine Bildbeschreibung erstellt.

<p><font color='black' size="5">
Aufbau
</font></p>

**RAGConfig**
- Zentrale Konfigurationsklasse
- Anpassbare Parameter (chunk_size, models, thresholds)

**RAGComponents**
- Container für alle System-Komponenten
- Text-Embeddings, CLIP-Model, LLMs, Collections

**Hauptfunktionen**
- Datensammlung
    - `init_rag_system()`: System-Initialisierung
    - `process_directory()`: Bulk-Import von Dateien
    - `add_text_document()`: Einzelnes Dokument hinzufügen
    - `add_image_with_description()`: Bild mit Auto-Beschreibung
- Abruf & Erweiterung
    - `search_texts()`: Text-Suche inkl. Bildbeschreibungen
    - `search_images()`: CLIP-basierte Bildsuche
    - `search_similar_images()`: Bild→Bild Ähnlichkeitssuche
    - `search_text_by_image()`: Bild→Text Suche
    - `multimodal_search()`: Erweiterte multimodale Suche

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/multimodal_rag_function_call.png" width="1100" alt="Avatar">




In [ ]:
# Import des Moduls
from multimodal_rag_modul import (
    init_rag_system,
    process_directory,
    multimodal_search,
    search_text_by_image,
    search_similar_images,
    add_text_document,
    add_image_with_description,
    get_system_status
)

# 3 | RAG initialisieren
---

In [ ]:
# Initialisierung
rag = init_rag_system()

In [ ]:
# Konfiguration abfragen
print(rag.config)

# Konfiguration ändern (Beispiel: chunk_size ändern)
# rag.config.chunk_size = 300

# 4 | Bilder/Texte in RAG übernehmen
---

In [ ]:
# Dokumente/Bilder laden & verarbeiten
process_directory(rag, './files', auto_describe_images=True)

# 5 | Suche Text -> Text/Bild
---

In [ ]:
result = multimodal_search(rag, "Was weisst Du über Cyborgs?")
mprint(result)

# 6 | Suche Bild -> Text/Bild
---

In [ ]:
# Bild → Bild: Finde visuell ähnliche Bilder
similar_images = search_similar_images(rag, "./zwei_roboter.png", k=5)
mprint("## 🖼️ Suche Bild → Bild")
mprint("---")
for img in similar_images:
    mprint(f"{img['filename']}: Ähnlichkeit: {img['similarity']}")

In [ ]:
# Bild → Text: Finde Textinformationen zu dem Bild
text_result = search_text_by_image(rag, "./zwei_roboter.png", k=5)
mprint(text_result)

# 7 | Aufgabe
---

<p><font color='black' size="5">
Teste das Multimodale RAG-System mit eigenen Queries
</font></p>

**Ziel**: Nutze das fertige multimodale RAG-System, um verschiedene Such-Operationen durchzuführen.



**Schritt 1: Einfache Queries testen**

Führe die folgenden Queries mit dem RAG-System aus und beobachte, welche Ergebnisse zurückgeliefert werden:

```python
# Query 1: Text → Text (Klassisches RAG)
result = multimodal_search(rag, "Wer war Alan Turing?")
mprint(result)

# Query 2: Text → Bild (Textanfrage findet Bilder)
result = multimodal_search(rag, "Zeige mir Bilder von Robotern")
mprint(result)

# Query 3: Bild → Bild (Finde ähnliche Bilder)
similar_images = search_similar_images(rag, "./files/apfel.jpg", k=3)
mprint("## 🖼️ Ähnliche Bilder:")
for img in similar_images:
    mprint(f"  • {img['filename']}: Ähnlichkeit {img['similarity']:.2f}")

# Query 4: Bild → Text (Finde Textinformationen zu einem Bild)
text_result = search_text_by_image(rag, "./files/hedra_cyborg.png", k=3)
mprint(text_result)
```



**Schritt 2: Eigene Queries erstellen**

Entwickle mindestens **3 eigene Queries**, die verschiedene Modalitäten nutzen:

**Beispiele für kreative Queries:**

- **Text→Text**: "Was ist der Unterschied zwischen Robotern und Cyborgs?"
- **Text→Bild**: "Finde alle futuristischen Bilder"
- **Bild→Bild**: Suche ähnliche Bilder zu `a_retro-futuristic_robot_dall_e.jpg`
- **Bild→Text**: Finde Textinformationen, die zum Apfel-Bild passen



**Schritt 3: Bonus-Challenge (Optional)**

Experimentiere mit den Suchparametern:

```python
# Ändere die Anzahl der Ergebnisse (k)
result = multimodal_search(rag, "Roboter", k=5)

# Vergleiche verschiedene Bilder für die Bild→Bild Suche
similar_1 = search_similar_images(rag, "./files/apfel.jpg", k=3)
similar_2 = search_similar_images(rag, "./files/hedra_cyborg.png", k=3)

# Finde heraus, welche Bilder am ähnlichsten zueinander sind
```

# A | Ähnlichkeitsmessung
---


<p><font color='black' size="5">
1. Text-Ähnlichkeit (semantisch)
</font></p>

  Embedding-Modell: OpenAI text-embedding-3-small

  Messmethode:
  - ChromaDB nutzt L2-Distanz (Euklidische Distanz) für Text-Embeddings
  - Werte: 0 = identisch, 2 = maximal entfernt
  - Konvertierung Distanz zu Ähnlichkeit: similarity = max(0, 1 - (score / 2))
  - Threshold: text_threshold: 1.2 (Zeile 51), Mindest-Ähnlichkeit: 0.3

  Verwendung in:
  - search_texts() - Text-Dokumente und Bildbeschreibungen durchsuchen
  - multimodal_search() - Kombinierte Suche

  ---

<p><font color='black' size="5">
2. Bild-Ähnlichkeit (visuell)
</font></p>

  Embedding-Modell: CLIP clip-ViT-B-32

  Messmethode:
  - ChromaDB nutzt Cosine-Distanz für Bild-Embeddings (Zeile 131: "hnsw:space": "cosine")
  - Werte: 0 = identisch, 2 = maximal entfernt
  - Konvertierung zu Ähnlichkeit:
  similarity = round(max(0, 1 - distance), 3)
  - Threshold: image_threshold: 0.8

  Verwendung in:
  - search_images() - Text → Bild Suche über CLIP   
  - search_similar_images() - Bild → Bild Suche  

  ---
  

<p><font color='black' size="5">
3. Cross-Modal-Retrieval (Text ↔ Bild)
</font></p>


  Methode: Indirekte Ähnlichkeit über Bildbeschreibungen

  Ablauf:
  1. Text → Bild: Text-Suche findet Bildbeschreibungen → verknüpfte Bilder werden abgerufen  
  2. Bild → Text: Bild-Suche findet ähnliche Bilder → deren Beschreibungen werden für semantische Textsuche verwendet

  Verknüpfung: Beide Collections sind über text_doc_id ↔ image_doc_id referenziert

  ---
  

<p><font color='black' size="5">
Zusammenfassung der Metriken:
</font></p>

  | Modalität | Embedding-Modell              | Distanzmetrik  | Threshold | Ähnlichkeitsbereich |
  |-----------|-------------------------------|----------------|-----------|---------------------|
  | Text      | OpenAI text-embedding-3-small | L2-Distanz     | 1.2       | 0.3 - 1.0           |
  | Bild      | CLIP ViT-B-32                 | Cosine-Distanz | 0.8       | 0.0 - 1.0           |

  Die Konvertierung 1 - (distance / 2) normalisiert beide Distanzmaße auf einen Ähnlichkeitswert von 0 bis 1, wobei 1 = maximale Ähnlichkeit bedeutet.